# Intégration de Semantic Kernel avec le serveur MCP OpenBnB

Ce notebook montre comment utiliser Semantic Kernel avec le serveur MCP OpenBnB réel pour rechercher des hébergements Airbnb en utilisant MCPStdioPlugin. Pour l'accès aux LLM, il utilise Azure AI Foundry. Pour configurer vos variables d'environnement, vous pouvez suivre la [Leçon de configuration](/00-course-setup/README.md)


## Importer les packages nécessaires


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Création de la connexion au plugin MCP

Nous allons nous connecter au [serveur MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) en utilisant MCPStdioPlugin. Ce serveur offre des fonctionnalités de recherche Airbnb via le package @openbnb/mcp-server-airbnb.


## Création du client

Dans cet exemple, nous utiliserons Azure AI Foundry pour accéder à notre LLM. Assurez-vous que vos variables d'environnement sont correctement configurées.


## Configuration de l'environnement

Configurez les paramètres d'Azure OpenAI. Assurez-vous d'avoir les variables d'environnement suivantes définies :
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Comprendre l'intégration OpenBnB MCP

Ce notebook se connecte au **véritable serveur MCP OpenBnB**, qui offre une fonctionnalité de recherche Airbnb réelle.

### Comment cela fonctionne :

1. **MCPStdioPlugin** : Utilise la communication via entrée/sortie standard avec le serveur MCP
2. **Package NPM réel** : Télécharge et exécute `@openbnb/mcp-server-airbnb` via npx
3. **Données en direct** : Renvoie des données réelles sur les propriétés Airbnb à partir de leurs API
4. **Découverte de fonctions** : L'agent découvre automatiquement les fonctions disponibles sur le serveur MCP

### Fonctions disponibles :

Le serveur MCP OpenBnB fournit généralement :
- **search_listings** - Recherche de propriétés Airbnb par emplacement et critères
- **get_listing_details** - Obtenir des informations détaillées sur des propriétés spécifiques
- **check_availability** - Vérifier la disponibilité pour des dates spécifiques
- **get_reviews** - Récupérer les avis sur les propriétés
- **get_host_info** - Obtenir des informations sur les hôtes des propriétés

### Prérequis :

- **Node.js** installé sur votre système
- **Connexion Internet** pour télécharger le package du serveur MCP
- **NPX** disponible (inclus avec Node.js)

### Tester la connexion :

Vous pouvez tester le serveur MCP manuellement en exécutant :
```bash
npx -y @openbnb/mcp-server-airbnb
```

Cela téléchargera et démarrera le serveur MCP OpenBnB, auquel Semantic Kernel se connectera ensuite pour obtenir des données Airbnb réelles.


## Exécution de l'Agent avec le serveur OpenBnB MCP

Nous allons maintenant exécuter l'Agent IA qui se connecte au serveur OpenBnB MCP pour rechercher de véritables logements Airbnb à Stockholm pour 2 adultes et 1 enfant. N'hésitez pas à modifier la liste `user_inputs` pour ajuster les critères de recherche.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Résumé  
Félicitations ! Vous avez réussi à créer un agent IA qui s'intègre à la recherche d'hébergements réels en utilisant le protocole Model Context Protocol (MCP) :

Technologies utilisées :  
- Semantic Kernel : pour construire des agents intelligents avec Azure OpenAI  
- Azure AI Foundry : pour les capacités de modèles de langage (LLM) et les complétions de chat  
- MCP (Model Context Protocol) : pour une intégration standardisée des outils  
- OpenBnB MCP Server : pour une recherche Airbnb réelle et fonctionnelle  
- Node.js/NPX : pour exécuter le serveur MCP externe  

Ce que vous avez appris :  
- Intégration MCP : Connecter des agents Semantic Kernel à des serveurs MCP externes  
- Accès aux données en temps réel : Rechercher des propriétés Airbnb réelles via des API en direct  
- Communication par protocole : Utiliser la communication stdio entre l'agent et le serveur MCP  
- Découverte de fonctions : Découvrir automatiquement les fonctions disponibles des serveurs MCP  
- Réponses en streaming : Capturer et enregistrer les appels de fonctions en temps réel  
- Rendu HTML : Formater les réponses de l'agent avec des tableaux stylisés et des affichages interactifs  

Prochaines étapes :  
- Intégrer des serveurs MCP supplémentaires (météo, vols, restaurants)  
- Construire un système multi-agents combinant les protocoles MCP et A2A  
- Créer des serveurs MCP personnalisés pour vos propres sources de données  
- Implémenter une mémoire de conversation persistante entre les sessions  
- Déployer l'agent sur Azure Functions avec orchestration de serveurs MCP  
- Ajouter des capacités d'authentification utilisateur et de réservation  

Principaux avantages de l'architecture MCP :  
- Standardisation : Protocole universel pour connecter des agents IA à des outils externes  
- Données en temps réel : Accès à des informations actualisées et en direct depuis divers services  
- Extensibilité : Intégration facile de nouvelles sources de données et outils  
- Interopérabilité : Fonctionne avec différents frameworks IA et plateformes d'agents  
- Séparation des préoccupations : Distinction claire entre la logique IA et l'accès aux données externes  



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
